## Imports necessários Glue Studio

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

# Comandos de configuração do GLUE

In [ ]:
# Comandos de configuração Glue-Spark
args = getResolvedOptions(sys.argv, ['JOB_NAME'])
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

# Carrega a base do catalogo de dados ou do s3 para o Glue

In [ ]:
# Carrega uma base de um catalogo de dados
base1 = glueContext.create_dynamic_frame.from_catalog(database = "my_database", table_name = "my_table", transformation_ctx = "datasource0")

# Carrega a base de um S3
base2 = glueContext.create_dynamic_frame.from_options("s3", {'paths': ["s3://my-bucket/my-file"]}, format="csv", transformation_ctx = "datasource0", format_options={'withHeader':True})

# Salvar arquivo no S3 e catalago de dados

In [ ]:
# A função getSink do GlueContext no AWS Glue é usada para configurar um conector que escreve os dados transformados para um destino específico.

write_s3_parquet = glueContext.getSink(
    path='s3://meu_caminho_s3'
    ,connection_type='S3'
    ,updateBehavior='UPDATE_IN_DATABASE'
    ,partitionKeys=['coluna_particao']
    ,compression='snappy'
    ,enableUpdateCatalog=True
    ,transformation_ctx='Write_S3_parquet'
)

write_s3_parquet.setCatalog(catalogDatabase='teste_database', catalogTableName='nomeTabela')

write_s3_parquet.setFormat('glueparquet')
write_s3_parquet.writeFrame(dataframe)

# Jeito mais simples - Esse método, eu ainda não testei

In [ ]:
# Salva a base no catalogo de dados
glueContext.write_dynamic_frame.from_catalog(frame = transformed_frame, database = "my_database", table_name = "my_table", transformation_ctx = "datasink")

# Salva a base no S3
glueContext.write_dynamic_frame.from_options(frame = transformed_frame, connection_type = "s3", connection_options = {"path": "s3://my-bucket/output/"}, format = "json", transformation_ctx = "datasink")

In [ ]:
# Finaliza o Glue-code
job.commit()